In [1]:
import json
import re
import random
import soundfile as sf
import numpy as np
import os
import mp

In [2]:
sr = 16000
pattern = r"<\|([\d.]+)\|>"

In [3]:
imda = []
with open('imda-whisper-word-timestamp.jsonl') as fopen:
    for l in fopen:
        imda.append(json.loads(l))
        
len(imda)

1633511

In [4]:
malay = []
with open('/home/husein/ssd3/pseudolabel-whisper-word-timestamp.jsonl') as fopen:
    for l in fopen:
        malay.append(json.loads(l))
        
len(malay)

4775944

In [5]:
!rm -rf /home/husein/ssd3/synthetic-context-switching-word-timestamp
!mkdir /home/husein/ssd3/synthetic-context-switching-word-timestamp

In [6]:
from tqdm import tqdm

selection = [malay, imda]
def loop(index):
    index, _ = index
    data = []
    for index_ in tqdm(index):
        chunk = []
        length = 0
        random_length = random.randint(10, 20)
        while length < random_length:
            x = random.choice(selection)
            s = random.choice(x)
            l = s['new_text'].split('<|')[-2].split('|>')[0]
            length += float(l)
            chunk.append(s)
        
        audio = [chunk[0]['audio_filename']]
        text = [chunk[0]['new_text'].replace('<|endoftext|>', '')]
        silents = []
        for i in range(1, len(chunk), 1):
            matches = re.findall(pattern, text[-1])
            last_t = float(matches[-1])
            silent_l = random.randint(int(sr * (50 / 1000)), int(sr * (500 / 1000)))
            t = chunk[i]['new_text'].replace('<|endoftext|>', '').split('<|transcribeprecise|>')[1]
            matches = re.findall(pattern, t)
            for m in matches:
                m_ = float(m)
                m_ += (silent_l / sr) + last_t
                t = t.replace(f'<|{m}|>', f'<|{m_:.2f}|>')
            silents.append(silent_l)
            text.append(t)
            audio.append(chunk[i]['audio_filename'])
        text = ''.join(text) + '<|endoftext|>'
        audios = []
        for no, a in enumerate(audio):
            if 'prepared-pseudolabel-chunks' in a:
                a = os.path.join('/home/husein/ssd3/', a)
            y, _ = sf.read(a)
            audios.append(y)

            if no < len(silents):
                audios.append(np.zeros((silents[0],)))
    
        try:
            audios = np.concatenate(audios)
            f = os.path.join('/home/husein/ssd3/synthetic-context-switching-word-timestamp', f'{index_}.mp3')
            sf.write(f, audios, sr)
            data.append({
                'new_text': text,
                'audio_filename': f,
            })
        except:
            pass
        
    return data

In [7]:
r = loop((range(10), 0))

100%|███████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 14.13it/s]


In [8]:
len(r)

10

In [9]:
r[-1]

{'new_text': "<|startoftranscript|><|en|><|transcribeprecise|><|0.00|> Milikul<|0.62|><|0.90|> Pavlik,<|1.46|><|1.82|> Low<|2.02|><|2.26|> Ing<|2.40|><|2.54|> Sing<|2.78|><|3.04|> and<|3.18|><|3.26|> Mohamad<|3.74|><|3.82|> Latif<|4.28|><|4.46|> jaga<|4.64|><|4.74|> Mana<|4.94|><|5.04|> lah<|5.12|><|5.20|> Allah<|5.38|><|5.44|> Ta'ala<|5.68|><|5.80|> ni<|5.86|><|5.92|> buat<|6.04|><|6.10|> Islam<|15.28|><|6.44|> So<|6.50|><|6.60|> jaga<|11.22|><|15.78|> semua<|7.02|><|7.08|> agama<|7.30|><|10.54|> Allah<|15.28|><|10.92|> Tuhan<|11.22|><|15.78|> kepada<|11.52|><|11.60|> Islam<|16.36|><|12.46|> Inna<|12.68|><|12.84|> dina<|17.52|><|13.18|> indallahil<|13.68|><|13.76|> Islam<|14.06|><|15.10|> Allah<|15.28|><|15.34|> buat<|15.44|><|15.50|> Islam<|15.72|><|15.78|> ni<|15.82|><|16.12|> Untuk<|16.36|><|16.44|> jaga<|16.66|><|16.80|> agama-agama<|17.40|><|17.52|> syirik<|17.74|><|17.84|> lain<|17.96|><|18.00|> sekali<|18.20|><|19.70|> Allah<|20.04|><|20.54|> Artinya<|20.86|><|20.96|> mengiktir

In [10]:
r = mp.multiprocessing(range(300000), loop, cores = 10)

100%|███████████████████████████████████████████████████████████████████████████████| 30000/30000 [1:18:28<00:00,  6.37it/s]


In [11]:
r[-1]

{'new_text': '<|startoftranscript|><|en|><|transcribeprecise|><|0.00|> a<|0.00|><|0.08|> very<|0.24|><|0.30|> well-known<|0.60|><|0.66|> figure<|0.88|><|2.26|> among<|2.44|><|2.54|> local<|2.76|><|2.84|> circles<|3.24|><|3.80|> and<|3.98|><|4.18|> overseas.<|4.64|><|4.88|> He<|4.92|><|5.04|> is<|5.10|><|5.18|> Professor<|5.60|><|5.70|> of<|5.74|><|5.78|> Political<|6.12|><|6.18|> Economy<|6.52|><|6.82|> at<|6.90|><|6.98|> the<|7.04|><|7.14|> Faculty<|7.50|><|7.60|> of<|7.64|><|7.72|> Economics<|8.16|><|8.24|> and<|8.30|><|8.34|> Administration,<|8.90|><|9.08|> Universiti<|9.40|><|9.46|> Umlair.<|9.74|><|11.08|> Dr<|11.30|><|11.36|> Gomez<|11.74|><|11.92|> specialises<|12.54|><|13.60|> in<|13.68|><|14.03|> the<|14.07|><|14.11|> way<|14.21|><|14.31|> out<|14.45|><|14.93|> and<|14.97|><|15.05|> come<|15.13|><|15.17|> back<|15.27|><|15.35|> in<|15.43|><|16.07|> and<|16.17|><|endoftext|>',
 'audio_filename': '/home/husein/ssd3/synthetic-context-switching-word-timestamp/299999.mp3'}

In [19]:
import copy

done = []
for r_ in r:
    r_ = copy.copy(r_)
    r_['audio_filename'] = r_['audio_filename'].split('ssd3/')[1]
    done.append(r_)

In [22]:
import pandas as pd

pd.DataFrame(done).to_parquet('synthetic-context-switching-word-timestamp.parquet')

In [23]:
!ls -lh synthetic-context-switching-word-timestamp.parquet

-rw-rw-r-- 1 husein husein 160M Feb  21 09:22 synthetic-context-switching-word-timestamp.parquet


In [25]:
from huggingface_hub import HfApi
api = HfApi()

api.upload_file(
    path_or_fileobj="synthetic-context-switching-word-timestamp.parquet",
    path_in_repo="data/synthetic_context_switching_word_timestamp-00000-of-00001.parquet",
    repo_id="malaysia-ai/whisper-STT",
    repo_type="dataset",
)

CommitInfo(commit_url='https://huggingface.co/datasets/malaysia-ai/whisper-STT/commit/7d0d965d100df84ffc415ea9d402e8ecb0c30339', commit_message='Upload data/synthetic_context_switching_word_timestamp-00000-of-00001.parquet with huggingface_hub', commit_description='', oid='7d0d965d100df84ffc415ea9d402e8ecb0c30339', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/malaysia-ai/whisper-STT', endpoint='https://huggingface.co', repo_type='dataset', repo_id='malaysia-ai/whisper-STT'), pr_revision=None, pr_num=None)